# abandoned for now (20201125)


- im just gonna precorrect my phenotypes for sex and generation and build a simple linear model with that, since i've been having convergence issues with the permutations

In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import copy

In [ ]:

#import matplotlib.pyplot as plt
#import seaborn as sns
#from collections import Counter


# import data

In [7]:

cross = pd.read_csv('20201123_cross.csv', low_memory=False)
cross = cross.drop([0,1])
cross = cross.reset_index()

cross = cross.replace('AA', -1)
cross = cross.replace('AB', 0)
cross = cross.replace('BB', 1)
cross = cross.replace('-', np.nan)

cross.GENERATION = ['F'+str(i) for i in cross.GENERATION]
cross.DAM = ['D'+str(i) for i in cross.DAM]

cross = cross.loc[cross.BW8.isna()==False]
cross.BW8 = cross.BW8.astype(int)
cross.SEX = cross.SEX.astype(int)


cnew = []
for i in cross.columns:
    if '-' in i:
        ni = 'mrk_'+'_'.join(i.split('-'))
    else:
        ni = i
    cnew.append(ni)
    
cross.columns = cnew


cross_markers = [i for i in cross.columns if i.startswith('mrk')]


# build model

In [8]:

def make_chicken_model(marker, data, pheno='BW8',):
    #drop all individuals where marker is missing
    subset = data.loc[data[marker].isna()==False] 
    
    md = smf.mixedlm("{pheno} ~ {marker}+ SEX".format(pheno=pheno, marker=marker), data=subset, groups='GENERATION')
    try:
        mdf = md.fit()
        return marker, mdf.pvalues[marker], mdf
    except:
        #print('linalgerror at marker {marker}'.format(marker=marker) ) 
        return marker, np.nan, None

In [32]:
def run_model(cross):
    cross_markers = [i for i in cross.columns if i.startswith('mrk')]
    pvals = {}
    for marker in cross_markers:
        m, p, model = make_chicken_model(marker=marker, data=cross)
        pvals[m]={'pvalue':p, 'model':model}
        
    pval2csv = []
    for key, item in pvals.items():
        mrk = "-".join(key.lstrip('mrk_').split('_'))
        pval = item['pvalue']
        if pvals[key]['model'] is not None:
            effectsize = pvals[key]['model'].params[key]
            se = pvals[key]['model'].bse_fe[key]
        else:
            effectsize = np.nan
            se = np.nan
        ni = [mrk, pval, effectsize, se]
        pval2csv.append(ni)
    
    
    return pval2csv

In [39]:
true_res = run_model(cross=cross)

In [41]:
#true_res

In [33]:
def run_permutation(i):
    crossloc = copy.deepcopy(cross)
    arrbw = np.array(crossloc['BW8'])
    np.random.shuffle(arrbw)
    crossloc['BW8'] = arrbw
    #return crossloc['BW8']
    pvals = run_model(cross = crossloc)
    return i,pvals

In [44]:
perm_dict = {}
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category='ConvergenceWarning')
    for i in range(3):
        j,k = run_permutation(i)
        perm_dict[j]=k
    

linalgerror at marker mrk_1_0
linalgerror at marker mrk_1_1
linalgerror at marker mrk_1_2
linalgerror at marker mrk_1_3
linalgerror at marker mrk_1_4
linalgerror at marker mrk_1_5
linalgerror at marker mrk_1_6
linalgerror at marker mrk_1_7
linalgerror at marker mrk_1_8
linalgerror at marker mrk_1_9
linalgerror at marker mrk_1_10
linalgerror at marker mrk_1_11
linalgerror at marker mrk_1_12
linalgerror at marker mrk_1_13
linalgerror at marker mrk_1_14
linalgerror at marker mrk_1_15
linalgerror at marker mrk_1_16
linalgerror at marker mrk_1_17
linalgerror at marker mrk_1_18
linalgerror at marker mrk_1_19
linalgerror at marker mrk_1_20
linalgerror at marker mrk_1_21
linalgerror at marker mrk_1_22
linalgerror at marker mrk_1_23
linalgerror at marker mrk_1_24
linalgerror at marker mrk_1_25
linalgerror at marker mrk_1_26
linalgerror at marker mrk_1_27
linalgerror at marker mrk_1_28
linalgerror at marker mrk_1_29
linalgerror at marker mrk_1_30
linalgerror at marker mrk_1_31
linalgerror at mar

In [47]:
perm_dict[1]

[['1-0', nan, nan, nan],
 ['1-1', nan, nan, nan],
 ['1-2', nan, nan, nan],
 ['1-3', nan, nan, nan],
 ['1-4', nan, nan, nan],
 ['1-5', nan, nan, nan],
 ['1-6', nan, nan, nan],
 ['1-7', 0.8860722313961815, 0.7752465503226613, 5.410862254908066],
 ['1-8', nan, nan, nan],
 ['1-9', nan, nan, nan],
 ['1-10', nan, nan, nan],
 ['1-11', nan, nan, nan],
 ['1-12', nan, nan, nan],
 ['1-13', nan, nan, nan],
 ['1-14', nan, nan, nan],
 ['1-15', nan, nan, nan],
 ['1-16', nan, nan, nan],
 ['1-17', nan, nan, nan],
 ['1-18', nan, nan, nan],
 ['1-19', nan, nan, nan],
 ['1-20', nan, nan, nan],
 ['1-21', nan, nan, nan],
 ['1-22', nan, nan, nan],
 ['1-23', nan, nan, nan],
 ['1-24', nan, nan, nan],
 ['1-25', nan, nan, nan],
 ['1-26', nan, nan, nan],
 ['1-27', nan, nan, nan],
 ['1-28', nan, nan, nan],
 ['1-29', nan, nan, nan],
 ['1-30', nan, nan, nan],
 ['1-31', nan, nan, nan],
 ['1-32', nan, nan, nan],
 ['1-33', nan, nan, nan],
 ['1-34', nan, nan, nan],
 ['1-35', nan, nan, nan],
 ['1-36', nan, nan, nan],
 ['1

In [38]:
pd.DataFrame(perm_dict[0])[1].min()

0.0020733072709951585

[828 383 623 ... 727 759 513]


In [1]:
from multiprocessing import Pool

In [ ]:
nproc =3

In [ ]:
with Pool(processes=nproc) as pool:
    pool.map_async

In [ ]:
perm_dict = {}